# 测试的工具函数

In [ ]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

import pandas as pd
from datasets import Dataset 
from ragas.metrics import answer_relevancy,answer_correctness,answer_correctness
from ragas import evaluate
data_path = "eval_reasult"
# gpt-4 ; gpt-3.5-turbo ; glm-3-turbo ; glm-4 ; rag

def eval_ar(q_lis:list,a_lis:list,t_lis):
    data_samples = {
        'question':q_lis,
        'answer':a_lis,
        'contexts':[[""] for i in range(len(q_lis))],
        'ground_truth':t_lis
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[answer_relevancy,answer_correctness])
    df = score.to_pandas()
    return df['answer_relevancy'].mean(),df['answer_correctness'].mean()

def process_Di(mt,data:list,_i = 1)->dict:
    # 返回模型的AC和AR
    dir ={f"AC{_i}":[],f'AR{_i}':[]}
    path = f"{data_path}\\{_i}"
    t_lis = [a['Answer'] for a in data]
    for _m in mt:
        m_data = f"{path}\eval_{_m}.csv"
        if _m == 'rag':
            m_data = "eval_reasult\\1\\new_eval_rag.csv"
        m_df = pd.read_csv(m_data).dropna(subset=['prediction'],axis=0)
        # _ac = m_df['answer_correctness'].mean()
        _ar,_ac = eval_ar(a_lis=list(m_df['prediction']),
                        q_lis=list(m_df['Question']),
                        t_lis=t_lis)
        print(f"{_m} 的平均相关性为 {_ar}")
        dir[f'AC{_i}'].append(_ac)
        print(f"{_m} 的平均正确率为 {_ac}")
        dir[f'AR{_i}'].append(_ar)
    return dir


# 检验文档完整性

In [3]:
# 检查评测数据中是否存在因为网络问题产生的缺失
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import pandas as pd
mt = ['ablat1_eval_rag_context']#'
def check(df,data):
    flag = False
    for _ in data:
        q = _['Question']
        if q not in list(df['Question']):
            print(q)
            return True
    return False


def remake_df(df:pd.DataFrame,data):
    r_df = pd.DataFrame([])
    for _ in data:
        q = _['Question']
        _df = df[df['Question']==q]
        _df = _df.iloc[-1,:]
        r_df = pd.concat([r_df,_df],axis=1)
    return r_df.T

# 检查因为未知原因出现的问答重复问题
for i in range(1,7):
    dataloader = DataLoader(f"_data\\testqa\\testdata_{i}.json")
    data = dataloader.load_json(dataloader.filepath)
    for _ in mt:
        _p = f"eval_reasult2\\{i}/{_}.csv"
        df = pd.read_csv(_p)
        if df.shape[0] != 50:
            print(f"{_} 的第{i}次数据",df.shape)
            #df.to_csv(f"eval_reasult\\{i}/eval_{_}_copy.csv")
            #r_df = remake_df(df,data)
            #print(r_df.shape)
            #r_df.to_csv(_p)
            #if check(df,data):
            #    print(f"{_} 的第{i}次数据,有缺陷")
        has_missing_values = df['prediction'].isnull().any()
        if has_missing_values:
            print(f"{_} 的第{i}次数据存在缺失 ")

# 评估

In [30]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

import pandas as pd
from datasets import Dataset 
from ragas.metrics import answer_relevancy,answer_correctness
from ragas import evaluate

# gpt-4 ; gpt-3.5-turbo ; glm-3-turbo ; glm-4 ; rag

def eval_ar(q_lis:list,a_lis:list,t_lis):
    data_samples = {
        'question':q_lis,
        'answer':a_lis,
        'contexts':[[""] for i in range(len(q_lis))],
        'ground_truth':t_lis
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[answer_relevancy,answer_correctness])
    df = score.to_pandas()
    return df['answer_relevancy'].mean(),df['answer_correctness'].mean()

def process_Di(mt,data:list,_i = 1)->dict:
    # 返回模型的AC和AR
    data_path = "eval_reasult2"
    path = f"{data_path}\\{_i}"
    
    dir ={f"AC{_i}":[],f'AR{_i}':[]}
    t_lis = [a['Answer'] for a in data]
    for _m in mt:
        m_data = f"{path}\{_m}.csv"
        m_df = pd.read_csv(m_data).dropna(subset=['prediction'],axis=0)
        # _ac = m_df['answer_correctness'].mean()
        _ar,_ac = eval_ar(a_lis=list(m_df['prediction']),
                        q_lis=list(m_df['Question']),
                        t_lis=t_lis)
        print(f"{_m} 的平均相关性为 {_ar}")
        dir[f'AC{_i}'].append(_ac)
        print(f"{_m} 的平均正确率为 {_ac}")
        dir[f'AR{_i}'].append(_ar)
    return dir


In [32]:
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import time
mt = ['ablat1_eval_rag_context']#'
d_lis = []
out_df = pd.DataFrame([],index=mt,columns=d_lis)
_p =  "eval_reasult2\\score0_default.csv"
out_df.to_csv(_p)
for i in range(1,7):
    out_df = pd.read_csv(_p,index_col=0)
    if i <1:
        continue
    print(out_df.shape)
    data = DataLoader.load_json(f"_data\\testqa\\testdata_{i}.json")
    d = process_Di(mt,_i=i,data=data)
    for _k in d.keys():
        out_df[_k] = d[_k]
    out_df.to_csv(_p)
    time.sleep(10)
out_df

(1, 0)


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat1_eval_rag_context 的平均相关性为 0.9746475074049553
ablat1_eval_rag_context 的平均正确率为 0.6484362398279808
(1, 2)


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat1_eval_rag_context 的平均相关性为 0.9602500228983634
ablat1_eval_rag_context 的平均正确率为 0.645604473674748
(1, 4)


...

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat1_eval_rag_context 的平均相关性为 0.9655998120151682
ablat1_eval_rag_context 的平均正确率为 0.6176519979389681
(1, 10)


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat1_eval_rag_context 的平均相关性为 0.9532023956695113
ablat1_eval_rag_context 的平均正确率为 0.6570481620794041


,AC1,AR1,AC2,AR2,AC3,AR3,AC4,AR4,AC5,AR5,AC6,AR6
ablat1_eval_rag_context,0.648436,0.974648,0.645604,0.96025,0.634834,0.948181,0.634636,0.93211,0.617652,0.9656,0.657048,0.953202


In [8]:
new_order = [f'AC{i}' for i in range(1,7)] +[f"AR{i}" for i in range(1,7)]
df = out_df[new_order]
# df.to_csv("eval_reasult2\\basellm_score2.csv")

,AC1,AC2,AC3,AC4,AC5,AC6,AR1,AR2,AR3,AR4,AR5,AR6
ablat1_eval_rag_context,0.762572,0.606089,0.654807,0.67465,0.630866,0.583318,0.956801,0.961551,0.948603,0.951893,0.962998,0.952345


对消融实验结果进行统计

In [2]:
# 检查评测数据中是否存在因为网络问题产生的缺失
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import pandas as pd

def check(df,data):
    flag = False
    for _ in data:
        q = _['Question']
        if q not in list(df['Question']):
            print(q)
            return True
    return False


mt = [f'ablat{i}_eval_rag_context' for i in range(1,5)] + [f'new_ablat{i}_eval_rag_context' for i in range(3,7)]

# 检查因为未知原因出现的问答重复问题
for i in range(1,7):
    dataloader = DataLoader(f"_data\\testqa\\testdata_{i}.json")
    data = dataloader.load_json(dataloader.filepath)
    for _ in mt:
        _p = f"eval_reasult2\\{i}/{_}.csv"
        df = pd.read_csv(_p)
        if df.shape[0] != 50:
            print(f"{_} 的第{i}次数据",df.shape)
            #df.to_csv(f"eval_reasult\\{i}/eval_{_}_copy.csv")
            #r_df = remake_df(df,data)
            #print(r_df.shape)
            #r_df.to_csv(_p)
            #if check(df,data):
            #    print(f"{_} 的第{i}次数据,有缺陷")
        has_missing_values = df['prediction'].isnull().any()
        if has_missing_values:
            print(f"{_} 的第{i}次数据存在缺失 ")

In [2]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

import pandas as pd
from datasets import Dataset 
from ragas.metrics import answer_relevancy,answer_correctness
from ragas import evaluate

def eval_ar(q_lis:list,a_lis:list,t_lis):
    data_samples = {
        'question':q_lis,
        'answer':a_lis,
        'contexts':[[""] for i in range(len(q_lis))],
        'ground_truth':t_lis
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[answer_relevancy,answer_correctness])
    df = score.to_pandas()
    return df['answer_relevancy'].mean(),df['answer_correctness'].mean()

def process_Di(mt,data:list,_i = 1)->dict:
    # 返回模型的AC和AR
    data_path = "eval_reasult2"
    path = f"{data_path}\\{_i}"
    
    dir ={f"AC{_i}":[],f'AR{_i}':[]}
    t_lis = [a['Answer'] for a in data]
    for _m in mt:
        m_data = f"{path}\{_m}.csv"
        m_df = pd.read_csv(m_data).dropna(subset=['prediction'],axis=0)
        # _ac = m_df['answer_correctness'].mean()
        _ar,_ac = eval_ar(a_lis=list(m_df['prediction']),
                        q_lis=list(m_df['Question']),
                        t_lis=t_lis)
        print(f"{_m} 的平均相关性为 {_ar}")
        dir[f'AC{_i}'].append(_ac)
        print(f"{_m} 的平均正确率为 {_ac}")
        dir[f'AR{_i}'].append(_ar)
    return dir


In [3]:
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import time
mt = [f'new_ablat{i}_eval_rag_context' for i in range(3,7)]
d_lis = []
out_df = pd.DataFrame([],index=mt,columns=d_lis)
_p =  "eval_reasult2\\score_tmp.csv"
out_df.to_csv(_p)
for i in range(1,7):
    out_df = pd.read_csv(_p,index_col=0)
    if i <1:
        continue
    print(out_df.shape)
    data = DataLoader.load_json(f"_data\\testqa2\\testdata_{i}.json")
    d = process_Di(mt,_i=i,data=data)
    for _k in d.keys():
        out_df[_k] = d[_k]
    out_df.to_csv(_p)
    time.sleep(10)
out_df

(4, 0)


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

new_ablat3_eval_rag_context 的平均相关性为 0.9229807914350774
new_ablat3_eval_rag_context 的平均正确率为 0.6359552370161132


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

new_ablat4_eval_rag_context 的平均相关性为 0.9391376768330777
new_ablat4_eval_rag_context 的平均正确率为 0.6540177195687843


...

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

new_ablat5_eval_rag_context 的平均相关性为 0.9460799665726085
new_ablat5_eval_rag_context 的平均正确率为 0.574270934109788


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

new_ablat6_eval_rag_context 的平均相关性为 0.9699164086302849
new_ablat6_eval_rag_context 的平均正确率为 0.5717752413434766


,AC1,AR1,AC2,AR2,AC3,AR3,AC4,AR4,AC5,AR5,AC6,AR6
new_ablat3_eval_rag_context,0.635955,0.922981,0.543278,0.959931,0.587066,0.890646,0.553695,0.884960,0.572301,0.933961,0.516164,0.947550
new_ablat4_eval_rag_context,0.654018,0.939138,0.584587,0.953725,0.612221,0.948943,0.546312,0.933754,0.561172,0.970727,0.557159,0.954067
new_ablat5_eval_rag_context,0.631364,0.979477,0.555832,0.958910,0.640381,0.945775,0.559870,0.949039,0.544213,0.940573,0.574271,0.946080
new_ablat6_eval_rag_context,0.634249,0.976259,0.565042,0.954594,0.611808,0.947576,0.586855,0.954866,0.604969,0.962887,0.571775,0.969916


In [4]:

new_order = [f'AC{i}' for i in range(1,7)] +[f"AR{i}" for i in range(1,7)]
out_df= out_df[new_order]
out_df.to_csv("eval_reasult2\\score2.csv")


In [5]:
df = pd.read_csv('eval_reasult2\\ablat_score.csv',index_col=0)
df = df.dropna()
df

,AC1,AC2,AC3,AC4,AC5,AC6,AR1,AR2,AR3,AR4,AR5,AR6
ablat1_eval_rag_context,0.775,0.611,0.657,0.671,0.627,0.596,0.974,0.962,0.945,0.950,0.964,0.954
ablat2_eval_rag_context,0.677,0.564,0.604,0.563,0.578,0.533,0.951,0.963,0.951,0.932,0.946,0.950
ablat3_eval_rag_context,0.623,0.581,0.587,0.567,0.548,0.548,0.950,0.933,0.842,0.880,0.953,0.908
ablat4_eval_rag_context,0.634,0.497,0.619,0.582,0.568,0.546,0.974,0.941,0.911,0.901,0.953,0.904
ablat5_eval_rag_context,0.663,0.544,0.631,0.570,0.605,0.599,0.974,0.958,0.955,0.955,0.958,0.964


In [9]:
df2 = pd.concat([df,out_df])
df2.to_csv('eval_reasult2\\ablat_score2.csv')

# 做消融实验的内容相关性测试

* 完整的模型 ==》 ablat1_eval_rag_context
* r2和IE，没有辅助模块和r1 ==》ablat4_eval_rag_context
* r1，没有辅助模块和r2、IE ==》 ablat5_eval_rag_context

In [26]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

import pandas as pd
from datasets import Dataset 
from ragas.metrics import answer_relevancy,answer_correctness,faithfulness,context_relevancy
from ragas import evaluate

def eval_ar(q_lis:list,a_lis:list,t_lis,contexts_lis:list):
    data_samples = {
        'question':q_lis,
        'answer':a_lis,
        'contexts':contexts_lis,
        'ground_truth':t_lis
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[faithfulness,context_relevancy])
    df = score.to_pandas()
    return df['faithfulness'].mean(),df['context_relevancy'].mean()

def process_Di(mt,data:list,_i = 1)->dict:
    # 返回模型的AC和AR
    data_path = "eval_reasult2"
    path = f"{data_path}\\{_i}"
    
    dir ={f"FAITH{_i}":[],f'CR{_i}':[]}
    t_lis = [a['Answer'] for a in data]
    for _m in mt:
        m_data = f"{path}\{_m}.csv"
        m_df = pd.read_csv(m_data).dropna(subset=['prediction'],axis=0)
        c_lis = []
        for _ in m_df['context']:
            c_lis.append([_])
        # _ac = m_df['answer_correctness'].mean()
        _F,_CR = eval_ar(a_lis=list(m_df['prediction']),
                        q_lis=list(m_df['Question']),
                        t_lis=t_lis,
                        contexts_lis=c_lis)
        print(f"{_m} 的忠实度为 {_F}")
        dir[f'FAITH{_i}'].append(_F)
        print(f"{_m} 的平均内容相关性为 {_CR}")
        dir[f'CR{_i}'].append(_CR)
    return dir


In [27]:
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import time
mt = [f'ablat{i}_eval_rag_context' for i in [1,4,5]]
d_lis = []
out_df = pd.DataFrame([],index=mt,columns=d_lis)
_p =  "eval_reasult2\\score_tmp.csv"
out_df.to_csv(_p)
for i in range(1,7):
    out_df = pd.read_csv(_p,index_col=0)
    if i <1:
        continue
    print(out_df.shape)
    data = DataLoader.load_json(f"_data\\testqa2\\testdata_{i}.json")
    d = process_Di(mt,_i=i,data=data)
    for _k in d.keys():
        out_df[_k] = d[_k]
    out_df.to_csv(_p)
    time.sleep(10)
out_df

(3, 0)


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat1_eval_rag_context 的忠实度为 0.9383778998778999
ablat1_eval_rag_context 的平均内容相关性为 0.12600928729087316


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

ablat4_eval_rag_context 的忠实度为 0.936396103896104
ablat4_eval_rag_context 的平均内容相关性为 0.11942419380317418


...

ablat4_eval_rag_context 的忠实度为 0.9650132275132276
ablat4_eval_rag_context 的平均内容相关性为 0.16724350852731085


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Task exception was never retrieved
future: <Task finished name='Task-5910' coro=<AsyncClient.aclose() done, defined at c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpx\_client.py:1967> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpx\_client.py", line 1977, in aclose
    await proxy.aclose()
  File "c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpx\_transports\default.py", line 365, in aclose
    await self._pool.aclose()
  File "c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpcore\_async\connection_pool.py", line 314, in aclose
    await connection.aclose()
  File "c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpcore\_async\http_proxy.py", line 332, in aclose
    await self._connection.aclose()
  File "c:\Users\donkey\.conda\envs\EDT\Lib\site-packages\httpcore\_async\http11.py", line 241, in aclose
    await self._network_stream.aclose()
  File "c:\Users\donkey\.c

ablat5_eval_rag_context 的忠实度为 0.9900488400488399
ablat5_eval_rag_context 的平均内容相关性为 0.6558809560392075


,FAITH1,CR1,FAITH2,CR2,FAITH3,CR3,FAITH4,CR4,FAITH5,CR5,FAITH6,CR6
ablat1_eval_rag_context,0.938378,0.126009,0.980008,0.089162,0.972948,0.099323,0.979429,0.085410,0.952563,0.069652,0.972883,0.133953
ablat4_eval_rag_context,0.936396,0.119424,0.973230,0.131292,0.937277,0.124115,0.958859,0.117230,0.915038,0.096209,0.965013,0.167244
ablat5_eval_rag_context,0.983697,0.576241,0.986739,0.298500,0.993738,0.394049,0.986253,0.337203,0.967975,0.561115,0.990049,0.655881


In [29]:
order  = [f'FAITH{i}' for i in range(1,7)] +[f'CR{i}' for i in range(1,7)]
out_df = out_df[order]
out_df.to_csv("eval_reasult2\\CR_score.csv")